## 优化模型参数
### 超参

是可以调整的参数，可以 控制 模型训练优化 的 过程

不同的超参可能会影响 模型训练 和 收敛速度

一般会定义以下用于训练的超参：
 - 训练轮次（epoch）：训练是遍历数据集的次数
 - 批次大小（batch size）：数据集进行分批读取训练，设定每个批次数据的大小
 - 学习率（learning rate）：学习率偏小 可能收敛过慢  学习率过大有可能会不收敛或者不可预测的结果
  

In [4]:
epoch = 5
batch_size = 64
learning_rate = 1e-3 

## 损失函数

损失函数用来评价模型的**预测值**与**真实值**偏离的程度

使用绝对误差损失函数L1Loss

还有很多Loss函数

In [5]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import dtype as mstype

loss = nn.L1Loss()
output_data = Tensor(np.array([[1,2,3],[2,3,4]]),dtype = mstype.float32)
target_data = Tensor(np.array([[0,2,5],[3,1,1]]),dtype = mstype.float32)
print(loss(output_data,target_data))

1.5


## 优化器
优化器用于 计算 和 更新 梯度

模型优化算法的选择直接关系到最终模型的的性能

效果不好，未必是特征和模型设计的问题，很有可能是优化器的问题

MindSpore所有优化器都封装在`Optimizer`对象中 

本次例子中我们使用SGD优化器

另有其他很多优化器在`mindspore.nn.optim`中

构建一个`Optimizer`对象，这个对象能够保持当前参数并给予计算得到的参数进行参数更新

他需要一个可以优化需要优化的参数（Variable对象）的迭代器 例如网络中所有可以训练的`parameter`

将`params`设置为`net.trainable_params()`即可

然后设置Optimizer的参数选项 学习率 权重衰减等

样例：
```python
from mindspore import nn
opyim=nn.SGD(params = net.trainable_params(),learning_rate=0.1,weight_decay = 0.0)
```


## 训练
一般有四个步骤
 - 定义神将网络
 - 构建数据集
 - 定义超参、损失函数、优化器
 - 输入训练轮次和数据集 进行训练
   

In [7]:
import numpy as np
import mindspore
import mindspore.nn as nn
from mindspore import Tensor,Model
from mindspore import  dtype as mstype
from mindspore.train.callback import LossMonitor
import mindspore.dataset.transforms.c_transforms as C
import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset as ds
import os


DATA_DIR = "./datasets/MNIST_Data/train"
# DATA_DIR = "./datasets/cifar-10-batches-bin/train"

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
#采样器
sampler = ds.SequentialSampler(num_samples=5)

# dataset = ds.Cifar100Dataset(DATA_DIR,sampler=sampler)

dataset = ds.MnistDataset(DATA_DIR,sampler=sampler) #这个是MNIST数据集
# dataset = ds.Cifar10Dataset(DATA_DIR,sampler=sampler) #这个是Cifar10数据集

class LeNet5(nn.Cell):
    def __init__(self,num_class=10,num_channel = 1):
    # 初始化网络
        super(LeNet5,self).__init__()
        # 定义所需要的运算
        self.conv1 = nn.Conv2d(num_channel,6,5,pad_mode='valid') # 卷积
        self.conv2 = nn.Conv2d(6,16,5,pad_mode='valid')
        self.fc1 = nn.Dense(16*5*5,120) # 全连接层
        self.fc2 = nn.Dense(120,84)
        self.fc3 = nn.Dense(84,num_class)
        self.max_pool2d = nn.MaxPool2d(kernel_size=2,stride=2)# 最大池化-降采样
        self.relu = nn.ReLU() # 激活函数
        self.flatten = nn.Flatten()# flatten 扁平的意思=> 将原来的高维数组换成只有 一行 的数组 列数是之前的各维度之积

    # 定义网络构建函数
    def construct(self,x):
        # 构建前向网络
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x    

#初始化网路
net = LeNet5()
for m in net.parameters_and_names():
    print(m)
    
#定义超参
epoch = 5
batch_size = 64
learning_rate = 1e-3
 
#构建数据集
sampler = ds.SequentialSampler(num_samples=128)
dataset = ds.MnistDataset(DATA_DIR,sampler = sampler)

#数据类型的转换
type_cast_op_image = C.TypeCast(mstype.float32)
type_cast_op_label = C.TypeCast(mstype.int32)

#数据序列读取方式
HWC2CHW = CV.HWC2CHW()

#构建数据集
dataset = dataset.map(operations=[type_cast_op_image,HWC2CHW],input_columns="image")
dataset = dataset.map(operations=type_cast_op_label,input_columns="label")
dataset = dataset.batch(batch_size)

#传入定义的超参
optim = nn.SGD(params=net.trainable_params(),learning_rate=learning_rate)
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True,reduction='mean') #交叉熵损失函数

model =Model(net,loss_fn=loss,optimizer=optim)
model.train(epoch=epoch,train_dataset=dataset,callbacks=LossMonitor())

[WARNING] ME(352:18168,MainProcess):2022-04-29-09:35:11.538.500 [mindspore\train\model.py:536] The CPU cannot support dataset sink mode currently.So the training process will be performed with dataset not sink.


('conv1.weight', Parameter (name=conv1.weight, shape=(6, 1, 5, 5), dtype=Float32, requires_grad=True))
('conv2.weight', Parameter (name=conv2.weight, shape=(16, 6, 5, 5), dtype=Float32, requires_grad=True))
('fc1.weight', Parameter (name=fc1.weight, shape=(120, 400), dtype=Float32, requires_grad=True))
('fc1.bias', Parameter (name=fc1.bias, shape=(120,), dtype=Float32, requires_grad=True))
('fc2.weight', Parameter (name=fc2.weight, shape=(84, 120), dtype=Float32, requires_grad=True))
('fc2.bias', Parameter (name=fc2.bias, shape=(84,), dtype=Float32, requires_grad=True))
('fc3.weight', Parameter (name=fc3.weight, shape=(10, 84), dtype=Float32, requires_grad=True))
('fc3.bias', Parameter (name=fc3.bias, shape=(10,), dtype=Float32, requires_grad=True))


ValueError: For 'MatMul', the input dimensions must be equal, but got 'x1_col': 256 and 'x2_row': 400. And 'x' shape [64, 256](transpose_a=False), 'y' shape [120, 400](transpose_b=True).